In [19]:
!apt update -y
!apt install gcc -y


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
43 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [22]:
%%writefile parallel_sort.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define SIZE 10000

void swap(int* a, int* b) {
    int t = *a;
    *a = *b;
    *b = t;
}

// ---------------- SEQUENTIAL BUBBLE SORT ----------------
void sequentialBubbleSort(int arr[], int n) {
    for (int i = 0; i < n-1; i++) {
        for (int j = 0; j < n-i-1; j++) {
            if (arr[j] > arr[j+1])
                swap(&arr[j], &arr[j+1]);
        }
    }
}

// ---------------- PARALLEL BUBBLE SORT ----------------
void parallelBubbleSort(int arr[], int n) {
    for (int i = 0; i < n-1; i++) {
        int start = i % 2;
        #pragma omp parallel for
        for (int j = start; j < n-1; j += 2) {
            if (arr[j] > arr[j+1]) {
                swap(&arr[j], &arr[j+1]);
            }
        }
    }
}

// ---------------- SEQUENTIAL MERGE SORT ----------------
void merge(int arr[], int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;
    int n2 = r - m;

    int* L = (int*) malloc(n1 * sizeof(int));
    int* R = (int*) malloc(n2 * sizeof(int));

    for (i = 0; i < n1; i++) L[i] = arr[l + i];
    for (j = 0; j < n2; j++) R[j] = arr[m + 1+ j];

    i = 0; j = 0; k = l;
    while (i < n1 && j < n2)
        arr[k++] = (L[i] <= R[j]) ? L[i++] : R[j++];
    while (i < n1) arr[k++] = L[i++];
    while (j < n2) arr[k++] = R[j++];

    free(L); free(R);
}

void sequentialMergeSort(int arr[], int l, int r) {
    if (l < r) {
        int m = (l + r)/2;
        sequentialMergeSort(arr, l, m);
        sequentialMergeSort(arr, m+1, r);
        merge(arr, l, m, r);
    }
}

// ---------------- PARALLEL MERGE SORT ----------------
void parallelMergeSort(int arr[], int l, int r) {
    if (l < r) {
        int m = (l + r)/2;
        #pragma omp parallel sections
        {
            #pragma omp section
            parallelMergeSort(arr, l, m);
            #pragma omp section
            parallelMergeSort(arr, m+1, r);
        }
        merge(arr, l, m, r);
    }
}

void copyArray(int src[], int dest[], int n) {
    for (int i = 0; i < n; i++) dest[i] = src[i];
}

void fillArray(int arr[], int n) {
    for (int i = 0; i < n; i++)
        arr[i] = rand() % 100000;
}

int main() {
    int n = SIZE;
    int original[SIZE], arr1[SIZE], arr2[SIZE], arr3[SIZE], arr4[SIZE];
    fillArray(original, n);

    copyArray(original, arr1, n);
    copyArray(original, arr2, n);
    copyArray(original, arr3, n);
    copyArray(original, arr4, n);

    double start, end;

    // Sequential Bubble Sort
    start = omp_get_wtime();
    sequentialBubbleSort(arr1, n);
    end = omp_get_wtime();
    printf("Sequential Bubble Sort Time: %f seconds\n", end - start);

    // Parallel Bubble Sort
    start = omp_get_wtime();
    parallelBubbleSort(arr2, n);
    end = omp_get_wtime();
    printf("Parallel Bubble Sort Time:   %f seconds\n", end - start);

    // Sequential Merge Sort
    start = omp_get_wtime();
    sequentialMergeSort(arr3, 0, n-1);
    end = omp_get_wtime();
    printf("Sequential Merge Sort Time:  %f seconds\n", end - start);

    // Parallel Merge Sort
    start = omp_get_wtime();
    parallelMergeSort(arr4, 0, n-1);
    end = omp_get_wtime();
    printf("Parallel Merge Sort Time:    %f seconds\n", end - start);

    return 0;
}


Writing parallel_sort.c


In [23]:
!gcc -fopenmp parallel_sort.c -o parallel_sort


In [24]:
!./parallel_sort


Sequential Bubble Sort Time: 0.363628 seconds
Parallel Bubble Sort Time:   0.232604 seconds
Sequential Merge Sort Time:  0.002437 seconds
Parallel Merge Sort Time:    0.009776 seconds
